# Assignment 2


In [1]:
import pickle
import pandas as pd
import urllib.request
import json
import math 
import ast
import re
from itertools import islice
from urllib.parse import quote

### Data

In [2]:
# 1. Load names/wiki links dictionary
def extractDataFiles(filename):
    marvel = pd.read_csv(filename +".csv")
    marvel_wikilinks = marvel.loc[:, 'WikiLink']
    marvel_characters = marvel_wikilinks[marvel_wikilinks.notna()].to_numpy()

    all_marvel = {}

    for name in marvel_characters:
        character = name.replace(" ", "_")
        all_marvel[character] = character

    return all_marvel

In [3]:
# 1. calling the functions
marvel = extractDataFiles('marvel')
dc = extractDataFiles('dc')

In [4]:
print(len(dc))
print(len(marvel))

518
1785


In [5]:
# 3. create URL with given character_wiki_link
# @returns NAME-> URL dict
def createURLs(data_dir): 
    
    url_query_list = {}
    
    for name,wiki in data_dir.items():        
        baseurl = "https://en.wikipedia.org/w/api.php?"
        action = "action=query"
        title = "titles=" + wiki
        content = "prop=revisions&rvprop=content"
        only_links = "prop=links&pllimit=max"
        dataformat ="format=json"
        
        query = "{}{}&{}&{}&{}&{}".format(baseurl, action, content, title,only_links, dataformat)
        url_query_list[name] = query
        
    
    return url_query_list

In [6]:
DC_url_query_directory_Links = createURLs(dc)
marvel_url_query_directory_Links = createURLs(marvel)

In [7]:
print(len(DC_url_query_directory_Links))
print(len(marvel_url_query_directory_Links))

518
1785


In [8]:
# 3.1 create URL with given character_wiki_link
# @returns NAME-> URL dict
def createURLsPages(data_dir): 
    
    url_query_list = {}
    
    for name,wiki in data_dir.items():        
        baseurl = "https://en.wikipedia.org/w/api.php?"
        action = "action=query"
        title = "titles=" + wiki
        content = "prop=revisions&rvprop=content"
        only_links = "&pllimit=max"
        dataformat ="format=json"
        
        query = "{}{}&{}&{}&{}&{}".format(baseurl, action, content, title,only_links, dataformat)
        url_query_list[name] = query
        
    
    return url_query_list

In [9]:
marvel_url_query_directory_pages = createURLsPages(marvel)
DC_url_query_directory_pages = createURLsPages(dc)

In [10]:

print(len(DC_url_query_directory_pages))


518


In [11]:
# 4. Send request

def sendWikiRequests(url_list):
    response_wiki_dir = {}

    for name,url in url_list.items():
        try:
            wikiresponse = urllib.request.urlopen(url)
            wikidata = wikiresponse.read()
            wikitext = wikidata.decode('utf-8')
        
            wiki_json = json.loads(wikitext)
            response_wiki_dir[name] = wiki_json
        except:
            pass
        
    return response_wiki_dir

In [12]:
DC_wiki_json_responses = sendWikiRequests(DC_url_query_directory_Links)
marvel_wiki_json_responses = sendWikiRequests(marvel_url_query_directory_Links)

In [13]:
marvel_wiki_responses_pages = sendWikiRequests(marvel_url_query_directory_pages)
dc_wiki_responses_pages = sendWikiRequests(DC_url_query_directory_pages)

In [14]:
def connectLinksPages(link_Directory,page_Directory):
    full_dict = {}
    
    for name, wiki in link_Directory.items():
        links = str(wiki)        
        pages = str(page_Directory[name])
        full = links + '***' + pages
        full_dict[name] = full
    return full_dict

In [ ]:
dcFUll = connectLinksPages(DC_wiki_json_responses, dc_wiki_responses_pages) 

In [ ]:
marvelFull = connectLinksPages(marvel_wiki_json_responses, marvel_wiki_responses_pages) 

In [ ]:
# Save to file
import os.path

def saveWikiJsonFiles(name_wiki,universe):
    for name,wiki in name_wiki.items():
        
        save_path = str(universe)+'/'
        name = name.replace('|','')
        completeName = os.path.join(save_path, name+".txt")      
        try:
            with open(completeName, "w") as text_file:
                    text_file.write(str(wiki))
        except:
            pass        

In [ ]:
saveWikiJsonFiles(marvelFull, 'marvel')

In [ ]:
saveWikiJsonFiles(dcFUll, 'dc')

In [ ]:
def readWikiJsonFiles(name_wiki,universe):
    
    name_wiki_json = {}
    
    for name,wiki in name_wiki.items():
        temp = []
        save_path = str(universe)+'/'
        completeName = os.path.join(save_path, name+".txt") 
        try:
            with open(completeName, "r") as text_file:
                f = text_file.read()
                name_wiki_json[name] = json.loads(json.dumps(f))
        except:
            pass
    
    return name_wiki_json


In [ ]:
DC_name_wiki = readWikiJsonFiles(dcFUll, 'dc')
marvel_name_wiki = readWikiJsonFiles(marvelFull, 'marvel')

In [ ]:
# preprocessing full pages / links
def splitLinksPages(text_file_list):
    
    name_wiki_dict = {}
    
    for name, wiki in text_file_list.items():
        x = wiki.split("***")    
        name_wiki_dict[name] = x
    
    return name_wiki_dict

In [ ]:
DC_name_wiki_processed = splitLinksPages(DC_name_wiki)
marvel_name_wiki_processed = splitLinksPages(marvel_name_wiki)


In [ ]:
beforeClean = len(DC_name_wiki_processed)
beforeClean2 = len(marvel_name_wiki_processed)

print('before clean ' , beforeClean) # le nomber is OKE based on the CSV provided
print('before clean2 ' , beforeClean2) # le nomber is OKE based on the CSV provided

In [ ]:

# 5. extractLinks for each title,
# @return array of names(links) to other pages
def extractNodeNeighbors(directory):
    
    linkList = []
    
    for i in range(len(directory)):
        ns = directory[i]['ns']
        if(int(ns) != 0):
            break
        else:
            title = directory[i]['title']
            linkList.append(title)
    
    return linkList

# 5.1 Check if site is redirect
def checkIfRedirect(directory):
    
    for i in range(len(directory)):
        ns = directory[i]['ns']
        if(int(ns) == 4):
            title = directory[i]['title']
            if "redirect" in title: 
                print('redirect')
                return 0
            else:
                return 1
        else:
            return 1

def wiki_json_preprocessing(directory):
    
    name_neighbors = {}
    
    for name,value_json in directory.items():
        try:
            value_json = value_json[0]
            value_json = ast.literal_eval(value_json)
        except:
            continue
        non_unique = value_json['query']['pages'].keys()
        non_unique = next(iter(non_unique))
        
        try:    
            link_dir = value_json['query']['pages'][non_unique]['links']
        except:
            continue
        #check for redirect
        
        try:
            if(checkIfRedirect(link_dir) == 0):
                print(directory[name] + ' DELETED')
                del directory[name]                
                continue
        except:
            pass
        else:
            node_neighbors = extractNodeNeighbors(link_dir)
                
        name_neighbors[name] = node_neighbors
        
    return name_neighbors      


In [ ]:
marvel_name_nodeLinks = wiki_json_preprocessing(marvel_name_wiki_processed)
DC_name_nodeLinks = wiki_json_preprocessing(DC_name_wiki_processed)

In [ ]:

print('after clean ', len(DC_name_nodeLinks))
print('after clean2 ', len(marvel_name_nodeLinks))


print('difference ', beforeClean - len(DC_name_nodeLinks))
print('difference2 ', beforeClean2 - len(marvel_name_nodeLinks))


**For each link you extract, check if the target is a character from your DC/Marvel lists. If yes, keep it. If no, discard it.**

In [ ]:
def checkFromTargetUniverse(character_nodes, universe):
    return list(set(character_nodes) & set(universe))

In [ ]:
# Marvel check


def removeRandomCharacters(name_links): 
    summ = 0
    
    for name, links in name_links.items():
        x_dc = checkFromTargetUniverse(links, dc)
        x_marvel = checkFromTargetUniverse(links, marvel)
        
        x_final = list(set().union(x_dc, x_marvel))
        summ += len(x_final)
        name_links[name] = x_final
    
    print('key -> value ::: sum of all edges in the values field ', summ)
    
    return name_links



In [ ]:
print('marvel')
marvel_name_nodeLinks = removeRandomCharacters(marvel_name_nodeLinks)
print('marvel nodes number : ', len((marvel_name_nodeLinks)))

print()
print('dc')
DC_name_nodeLinks = removeRandomCharacters(DC_name_nodeLinks)
print('DC nodes number ', len((DC_name_nodeLinks)))


#### Use a NetworkX DiGraph to store the network. As noted above, remember to store the properties of the nodes (i.e. from which universe they hail).


In [ ]:
# IMPORTS
import networkx as nx

In [ ]:
# G.add_node('abc', dob=1185, pob='usa', dayob='monday')

G = nx.DiGraph()

#marvel
for name, connections in marvel_name_nodeLinks.items():
    G.add_node(name, universe = 'marvel')

#dc
for name, connections in DC_name_nodeLinks.items():
    G.add_node(name, universe = 'dc')

for name, connections in marvel_name_nodeLinks.items(): 
    for i in range(len(connections)):
        G.add_edges_from(name, connections[i])

### Basic Stats

In [ ]:
# Get basic stats, mean, median etc... (once this runs and a proper graph is constructed :)
print(G.number_of_edges())
print(G.number_of_nodes())

In [ ]:
# Get stats for in- and out- degree distribution
import collections
import matplotlib.pyplot as plt

print(G)

# All degrees
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
degree_count = collections.Counter(degree_sequence)
degrees, counts = zip(*degree_count.items())


top_five = sorted(G.degree(), key=lambda x:x[1], reverse=True)[:50]
print("Top five ", top_five)
print()

# In degrees
degree_sequence = sorted([d for n, d in G.in_degree()], reverse=True)
degree_count = collections.Counter(degree_sequence)
in_degrees, in_counts = zip(*degree_count.items())

# Out degrees
degree_sequence = sorted([d for n, d in G.out_degree()], reverse=True)
degree_count = collections.Counter(degree_sequence)
out_degrees, out_counts = zip(*degree_count.items())

plt.plot(degrees, counts, 'ro')
plt.show()

The number of nodes is...

In [ ]:
subgraphs = [G.subgraph(c) for c in nx.weakly_connected_components(G)]

# Probably not necessary to iterate
GCC = subgraphs[0]
print('aaa', type(GCC))
print(GCC)
for subgraph in subgraphs[1:]:
    if subgraph.number_of_nodes() > GCC.number_of_nodes():
        GCC = subgraph

print(GCC)
        
 # Universe attribute
attributes_dict = {}
for i in marvel_name_nodeLinks.keys():
    attributes_dict[i] = "marvel"

for i in DC_name_nodeLinks.keys():
    attributes_dict[i] = "dc"

nx.set_node_attributes(GCC, attributes_dict, name="universe")

GCC_undirected = GCC.to_undirected()


In [ ]:
pos = nx.kamada_kawai_layout(GCC_undirected)

node_colors = []

for node in list(GCC_undirected.nodes()):
    if GCC_undirected[node]['universe'] == 'dc':
        node_color = "#FF0000"
    else:
        node_color = "FFFF00"
    node_colors.append(node_color)    

plt.figure(3,figsize=(12,16)) 
nx.draw(GCC_undirected, pos=pos, node_size=20, node_color = node_colors)



Plotting the network...

In [ ]:
node_colors = []

for node in list(GCC_undirected.nodes()):
    if GCC_undirected[node]['universe'] == 'dc':
        node_color = "#FF0000"
    else:
        node_color = "FFFF00"
    node_colors.append(node_color)    


In [ ]:
from fa2 import ForceAtlas2

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=2.0,
                        strongGravityMode=False,
                        gravity=1.0,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(GCC_undirected, pos=None, iterations=2000)

In [ ]:
degree_sequence = [d for n, d in GCC_undirected.degree()]
degree_count = collections.Counter(degree_sequence)

offset = 10
scale = 200
range = float(max(degree_sequence) - min(degree_sequence))
node_sizes = [offset + scale * degree / range for degree in degree_sequence]

nx.draw(GCC_undirected, pos=positions, node_size=node_sizes, width=0.15, node_color=node_colors, edgecolors="#000000", linewidths=1)

### Communities

### Sentiment